In [ ]:
!pip install quimb qiskit qiskit-aer qiskit-ibm-runtime numpy scipy matplotlib qutip

In [ ]:
!pip install --upgrade qiskit-ibm-runtime

### **Step 1: Defining the parameters and importing the libraries**

In [ ]:
import numpy as np
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qutip import Qobj, krylovsolve
import matplotlib.pyplot as plt
import time

# Parameters
N = 16  # Grid points (1D for Cole-Hopf)
dx = 1.0 / (N - 1)
nu = 0.1
dt = 0.0005
u_L = 1.0
u_R = 0.0
x = np.linspace(0, 1, N)
n_qubits = int(np.ceil(np.log2(N)))  # 4 qubits
SHOTS = 8192
fixed_x_idx = 8
use_derivative_oracle = False

# Time step sets
time_step_sets = [
    [0.0, 0.005, 0.01],
    [0.0, 0.0025, 0.005, 0.0075, 0.01],
    [0.0, 0.002, 0.004, 0.006, 0.008, 0.01]
]

Step 2: Setting up the backend and defining the Cole-Hopf transform

In [ ]:
# Step 2: Cole-Hopf Transform
u_0 = np.sin(np.pi * x)
integral_u = np.zeros(N)
for i in range(1, N):
    integral_u[i] = integral_u[i-1] + 0.5 * dx * (u_0[i-1] + u_0[i])
psi_0 = np.exp(-integral_u / (2 * nu))
psi_0 = psi_0 / np.linalg.norm(psi_0)
psi_0_padded = np.pad(psi_0, (0, 2**n_qubits - N), mode='constant')
psi_0_padded = psi_0_padded / np.linalg.norm(psi_0_padded)

# Backend setup
try:
    service = QiskitRuntimeService(channel="ibm_cloud", token="crawIW-p8bwcIk25BlpY1u_Ui2jEPt9NKQ_9tZYEIevf", instance="crn:v1:bluemix:public:quantum-computing:us-east:a/00b9ddaacd6b43b8a0403feb24ecf512:788e6594-fce2-41e0-973a-9d6c14c4c0ec::")
    backend = service.backend("ibm_torino")
except Exception as e:
    print(f"Error accessing ibm_torino: {e}")
    backend = AerSimulator()
    print("Using AerSimulator as fallback")

try:
    noise_model = NoiseModel.from_backend(backend)
    print("Noise model applied successfully")
except Exception as e:
    noise_model = None
    print(f"No noise model; using ideal simulator: {e}")

# Transpilation setup
layout = list(range(n_qubits))  # 4 qubits for Noisy/ZNE
pm = generate_preset_pass_manager(optimization_level=1, backend=backend)

Step 3: Creating the Trotter circuit

In [ ]:
# Step 3: Trotter Circuit
def create_trotter_circuit(scale_factor, t, dt, n_qubits, psi_0_padded, noise_scale=1):
    qr = QuantumRegister(n_qubits, 'q')
    cr = ClassicalRegister(n_qubits, 'meas')
    qc = QuantumCircuit(qr, cr)
    qc.initialize(psi_0_padded, qr)
    steps = max(1, int(t / dt))
    for _ in range(steps):
        for i in range(n_qubits-1):
            for _ in range(scale_factor * noise_scale):
                qc.rxx(2 * nu * dt / dx**2, i, i+1)
            qc.barrier()
    qc.measure(qr, cr)
    return qc

Step 4: ZNE Implementation

In [ ]:
# Step 4: ZNE Implementation
def run_zne_circuit(t, dt, n_qubits, psi_0_padded, simulator, noise_model, pm, shots):
    noise_scales = [1, 3]
    results = []
    qc_transpiled = None
    for scale in noise_scales:
        qc = create_trotter_circuit(1, t, dt, n_qubits, psi_0_padded, noise_scale=scale)
        qc_transpiled = pm.run(qc)
        job = simulator.run(qc_transpiled, shots=shots, noise_model=noise_model)
        counts = job.result().get_counts()
        psi_t = np.sqrt(np.array([counts.get(bin(i).replace('0b', '').zfill(n_qubits), 0) for i in range(N)])) / np.sqrt(shots)
        psi_t = psi_t / np.linalg.norm(psi_t) if np.linalg.norm(psi_t) > 0 else psi_t
        u_t = np.zeros(N)
        for i in range(1, N-1):
            u_t[i] = -2 * nu * (psi_t[i+1] - psi_t[i-1]) / (2 * dx * max(psi_t[i], 1e-10))
        u_t[0] = u_L
        u_t[-1] = u_R
        results.append(u_t)
    u_zne = 1.5 * results[0] - 0.5 * results[1]
    u_zne = np.clip(u_zne, -1, 2)
    return u_zne, qc_transpiled

Step 5: Define the tridiagonal Laplacian matrix

In [ ]:
# Step 5: Laplacian Matrix for QuTiP and Classical
def create_laplacian_matrix(N, dx, nu, u_L, u_R):
    L = np.zeros((N, N))
    for i in range(N):
        L[i, i] = -2 / dx**2
        if i > 0:
            L[i, i-1] = 1 / dx**2
        if i < N-1:
            L[i, i+1] = 1 / dx**2
    L[0, 0] = L[-1, -1] = 0  # Boundary conditions
    return L

Step 6: Execute the quantum circuit and the classical Krylov solver and note down the resouces used

In [ ]:
# Step 6: Process Time Step Sets
simulator = AerSimulator()

for time_steps in time_step_sets:
    u_noisy_steps = []
    u_zne_steps = []
    u_t_steps = []
    u_classical_steps = []
    circuit_depths_noisy = []
    circuit_depths_zne = []
    two_qubit_gates_noisy = []
    two_qubit_gates_zne = []
    l2_errors_qutip = []
    shock_positions_noisy = []
    shock_positions_zne = []
    shock_positions_qutip = []
    dissipation_rates_noisy = []
    dissipation_rates_zne = []
    dissipation_rates_qutip = []
    u_classical_fixed_points = []
    wall_times = []
    start_time = time.time()

    # QuTiP and Classical for 6-step set
    if len(time_steps) == 6:
        psi_0_vector = psi_0  # Use psi_0 from Cole-Hopf
        L_matrix = create_laplacian_matrix(N, dx, nu, u_L, u_R)
        H_qutip = Qobj(-1j * nu * L_matrix)  # Qobj for krylovsolve

        for t in time_steps:
            steps = int(t / dt) if t > 0 else 0
            tlist = np.linspace(0, t, steps + 1) if t > 0 else [0]

            # QuTiP Krylov
            if t > 0:
                result = krylovsolve(H_qutip, Qobj(psi_0_vector), tlist, krylov_dim=10)
                psi_t = np.real(result.states[-1].full().flatten())
            else:
                psi_t = psi_0_vector

            u_t = np.zeros(N)
            if use_derivative_oracle:
                u_t[1:N-1] = -2 * nu * psi_t[1:N-1] / (psi_0[1:N-1] + 1e-10)
            else:
                for i in range(1, N-1):
                    u_t[i] = -2 * nu * (psi_t[i+1] - psi_t[i-1]) / (2 * dx * (psi_t[i] + 1e-10))
            u_t[0] = u_L
            u_t[-1] = u_R
            u_t_steps.append(u_t)

            # Classical Solution (Benchmark)
            if t == 0:
                psi_classical = psi_0
            else:
                result = krylovsolve(H_qutip, Qobj(psi_0), tlist, krylov_dim=10)
                psi_classical = np.real(result.states[-1].full().flatten())
            u_classical = np.zeros(N)
            for i in range(1, N-1):
                u_classical[i] = -2 * nu * (psi_classical[i+1] - psi_classical[i-1]) / (2 * dx * (psi_classical[i] + 1e-10))
            u_classical[0] = u_L
            u_classical[-1] = u_R
            u_classical_steps.append(u_classical)
            u_classical_fixed_points.append(u_classical[fixed_x_idx])

            l2_error = np.linalg.norm(u_t - u_classical)
            l2_errors_qutip.append(l2_error if l2_error < 1e3 else np.nan)

            shock_idx = np.argmin(np.abs(u_t - 0.5)) if np.any(np.abs(u_t - 0.5) < 0.5) else N//2
            shock_pos = x[shock_idx]
            shock_positions_qutip.append(shock_pos)

            grad_u = np.gradient(u_t, dx)
            dissipation = nu * np.sum(grad_u**2) * dx
            dissipation_rates_qutip.append(dissipation)

    # Noisy Quantum and ZNE
    for t in time_steps:
        qc = create_trotter_circuit(1, t, dt, n_qubits, psi_0_padded, noise_scale=1)
        qc_transpiled = pm.run(qc)
        circuit_depths_noisy.append(qc_transpiled.depth())
        ops = qc_transpiled.count_ops()
        two_qubit_count = sum(ops.get(gate, 0) for gate in ['rxx', 'cx', 'cnot'])
        two_qubit_gates_noisy.append(two_qubit_count)
        print(f"t={t}: Noisy Ops = {ops}, Two-qubit gates = {two_qubit_count}")
        job_sim = simulator.run(qc_transpiled, shots=SHOTS, noise_model=noise_model)
        counts = job_sim.result().get_counts()
        psi_t = np.sqrt(np.array([counts.get(bin(i).replace('0b', '').zfill(n_qubits), 0) for i in range(N)])) / np.sqrt(SHOTS)
        psi_t = psi_t / np.linalg.norm(psi_t) if np.linalg.norm(psi_t) > 0 else psi_t
        u_t = np.zeros(N)
        for i in range(1, N-1):
            u_t[i] = -2 * nu * (psi_t[i+1] - psi_t[i-1]) / (2 * dx * max(psi_t[i], 1e-10))
        u_t[0] = u_L
        u_t[-1] = u_R
        u_noisy_steps.append(u_t)

        shock_pos = x[np.argmax(np.abs(np.diff(u_t) / dx))]
        u_x = np.diff(u_t) / dx
        dissipation_rate = nu * np.sum(u_x**2) * dx
        shock_positions_noisy.append(shock_pos)
        dissipation_rates_noisy.append(dissipation_rate)

        u_zne, qc_zne_transpiled = run_zne_circuit(t, dt, n_qubits, psi_0_padded, simulator, noise_model, pm, SHOTS)
        u_zne_steps.append(u_zne)
        circuit_depths_zne.append(qc_zne_transpiled.depth())
        ops_zne = qc_zne_transpiled.count_ops()
        two_qubit_count_zne = sum(ops_zne.get(gate, 0) for gate in ['rxx', 'cx', 'cnot','cz'])
        two_qubit_gates_zne.append(two_qubit_count_zne)

        shock_pos_zne = x[np.argmax(np.abs(np.diff(u_zne) / dx))]
        u_x_zne = np.diff(u_zne) / dx
        dissipation_rate_zne = nu * np.sum(u_x_zne**2) * dx
        shock_positions_zne.append(shock_pos_zne)
        dissipation_rates_zne.append(dissipation_rate_zne)

    wall_times.append(time.time() - start_time)

Step 7: Plot the u values for all the time steps, the circuit depth, and Dissipation rates

In [ ]:
    # Plotting: 6 graphs for 6-step set, each with Noisy, ZNE, and Classical
    if len(time_steps) == 6:
        for i, t in enumerate(time_steps):
            plt.figure(figsize=(10, 6))
            plt.plot(x, u_noisy_steps[i], marker='o', linestyle='-', label='Noisy Quantum')
            plt.plot(x, u_zne_steps[i], marker='s', linestyle='--', label='ZNE')
            plt.plot(x, u_classical_steps[i], marker='^', linestyle=':', label='Classical')
            plt.xlabel('x')
            plt.ylabel('u(x,t)')
            plt.title(f'Burgers\' Solution at t={t:.3f} (Noisy, ZNE, Classical)')
            plt.legend()
            plt.grid(True)
            plt.show()

        # Dissipation Rate Plot
        plt.figure(figsize=(10, 6))
        plt.plot(time_steps, dissipation_rates_noisy, marker='o', linestyle='-', label='Noisy Quantum')
        plt.plot(time_steps, dissipation_rates_zne, marker='s', linestyle='--', label='ZNE')
        plt.plot(time_steps, dissipation_rates_qutip, marker='^', linestyle=':', label='QuTiP')
        plt.xlabel('Time (t)')
        plt.ylabel('Dissipation Rate')
        plt.title('Dissipation Rate vs Time (6 Time Steps)')
        plt.legend()
        plt.grid(True)
        plt.show()

    # Circuit Depth Plot
    plt.figure(figsize=(10, 6))
    plt.plot(time_steps, circuit_depths_noisy, marker='o', label='Noisy Quantum Depth')
    plt.plot(time_steps, circuit_depths_zne, marker='s', label='ZNE Depth')
    plt.xlabel('Time (t)')
    plt.ylabel('Circuit Depth')
    plt.title(f'Circuit Depth vs Time ({len(time_steps)} Time Steps)')
    plt.legend()
    plt.grid(True)
    plt.show()

Step 8: Print the L2 errors, Shock Front Positions, Wall clock times and all the other resources used

In [ ]:
    # Print Metrics
    print(f"\nTime Steps: {time_steps}")
    if len(time_steps) == 6:
        print(f"L2 Errors (QuTiP vs Classical): {l2_errors_qutip}")
    print(f"Shock Front Positions (Noisy): {shock_positions_noisy}")
    print(f"Shock Front Positions (ZNE): {shock_positions_zne}")
    if len(time_steps) == 6:
        print(f"Shock Front Positions (QuTiP): {shock_positions_qutip}")
    print(f"Dissipation Rates (Noisy): {dissipation_rates_noisy}")
    print(f"Dissipation Rates (ZNE): {dissipation_rates_zne}")
    if len(time_steps) == 6:
        print(f"Dissipation Rates (QuTiP): {dissipation_rates_qutip}")
    print(f"Classical Fixed Points (x={x[fixed_x_idx]}): {u_classical_fixed_points}")
    print(f"Wall-clock time: {wall_times[-1]:.2f} seconds")
    print(f"Quantum Metrics (Noisy):")
    print(f"  Qubits: {n_qubits}")
    print(f"  Circuit Depths: {circuit_depths_noisy}")
    print(f"  Two-qubit Gates: {two_qubit_gates_noisy}")
    print(f"  T-count: {qc_transpiled.count_ops().get('t', 0)}")
    print(f"Quantum Metrics (ZNE):")
    print(f"  Qubits: {n_qubits}")
    print(f"  Circuit Depths: {circuit_depths_zne}")
    print(f"  Two-qubit Gates: {two_qubit_gates_zne}")
    print(f"  T-count: {qc_zne_transpiled.count_ops().get('t', 0)}")